# Getting Started with AutoGen

## Welcome to AutoGen!

AutoGen is a programming framework for building multi-agent AI applications. In this quickstart notebook, we'll:

1. Install AutoGen
2. Create your first agent
3. Build a simple two-agent conversation
4. Add tools and function calling
5. Stream agent responses

Let's get started!

## 1. Installation

First, let's install the required packages:

In [ ]:
# Install AutoGen AgentChat and Extensions
!pip install -U "autogen-agentchat" "autogen-ext[openai]"

## 2. Setting up API Keys

AutoGen works with various LLM providers. For this tutorial, we'll use OpenAI:

In [1]:
import os
from getpass import getpass

# Set your OpenAI API key
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

## 3. Creating Your First Agent

Let's create a simple assistant agent:

In [2]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Initialize the model client
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    # api_key=os.environ["OPENAI_API_KEY"],  # Optional, will use env var
)

# Create an assistant agent
assistant = AssistantAgent(
    name="helpful_assistant",
    model_client=model_client,
    system_message="You are a helpful AI assistant. Be concise and informative.",
)

print("Agent created successfully!")

Agent created successfully!


## 4. Running Your First Agent Task

Now let's run a simple task with our agent:

In [4]:
# Run a simple task
result = await assistant.run(task="What is a large language model?")

# Print the result
print("Agent Response:")
print("-" * 50)
for message in result.messages:
    print(f"{message.source}: {message.content}")

Agent Response:
--------------------------------------------------
user: What is a large language model?
helpful_assistant: A large language model (LLM) is a type of artificial intelligence that is trained on vast amounts of text data to understand and generate human language. Key characteristics of LLMs include:

1. **Scale**: LLMs have a large number of parameters (often billions or trillions), which allows them to capture complex patterns and nuances in language.

2. **Training**: They are trained using machine learning techniques, particularly deep learning, on diverse datasets comprising books, articles, websites, and more.

3. **Natural Language Understanding and Generation**: LLMs can perform tasks such as text completion, summarization, translation, conversation, and question-answering.

4. **Contextual Awareness**: They can generate coherent and contextually relevant responses based on the input they receive.

5. **Few-Shot and Zero-Shot Learning**: LLMs can often perform task

## 5. Adding Tools to Your Agent

Agents become more powerful when they can use tools. Let's add a simple tool:

In [5]:
# Define a simple tool
async def get_weather(city: str) -> str:
    """Get the weather for a given city.
    
    Args:
        city: The name of the city
    
    Returns:
        A string describing the weather
    """
    # This is a mock function for demonstration
    weather_data = {
        "New York": "Sunny, 72°F",
        "London": "Cloudy, 59°F",
        "Tokyo": "Rainy, 65°F",
        "Paris": "Partly cloudy, 68°F",
    }
    return weather_data.get(city, f"Weather data not available for {city}")

# Create an agent with tools
weather_agent = AssistantAgent(
    name="weather_assistant",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a weather assistant. Use the get_weather tool to provide weather information.",
)

# Run a task that uses the tool
result = await weather_agent.run(task="What's the weather like in New York and Paris?")

print("Weather Agent Response:")
print("-" * 50)
for message in result.messages:
    if message.content:  # Only print messages with content
        print(f"{message.source}: {message.content}")

Weather Agent Response:
--------------------------------------------------
user: What's the weather like in New York and Paris?
weather_assistant: [FunctionCall(id='call_HcNdF4GyDJrmWdm07zQQxPIZ', arguments='{"city": "New York"}', name='get_weather'), FunctionCall(id='call_D9nIoTOwZ1kesy3ycPqSwHjX', arguments='{"city": "Paris"}', name='get_weather')]
weather_assistant: [FunctionExecutionResult(content='Sunny, 72°F', name='get_weather', call_id='call_HcNdF4GyDJrmWdm07zQQxPIZ', is_error=False), FunctionExecutionResult(content='Partly cloudy, 68°F', name='get_weather', call_id='call_D9nIoTOwZ1kesy3ycPqSwHjX', is_error=False)]
weather_assistant: Sunny, 72°F
Partly cloudy, 68°F


## 6. Creating a Two-Agent Conversation

Let's create a conversation between two agents:

In [7]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import MaxMessageTermination

# Create a writer agent
writer = AssistantAgent(
    name="writer",
    model_client=model_client,
    system_message="You are a creative writer. Write short, engaging content.",
)

# Create a critic agent
critic = AssistantAgent(
    name="critic",
    model_client=model_client,
    system_message="You are a constructive critic. Provide brief, helpful feedback to improve the writing. After 2 rounds of feedback, say 'APPROVED' if satisfied.",
)

# Create a team with both agents
team = RoundRobinGroupChat(
    participants=[writer, critic],
    termination_condition=MaxMessageTermination(max_messages=6),
)

# Run the team
result = await team.run(task="Write a haiku about artificial intelligence")

print("\nConversation:")
print("=" * 50)
for message in result.messages:
    print(f"\n{message.source}:")
    print(message.content)


Conversation:

user:
Write a haiku about artificial intelligence

writer:
Silicon whispers,  
Thoughts woven in circuits' dance,  
Dreams of code and light.

critic:
This haiku presents a beautiful imagery of artificial intelligence. To enhance it further, consider refining the emotional connection. Perhaps emphasize the impact of AI on human experience or its potential. 

Additionally, ensure the syllable count adheres strictly to the 5-7-5 structure. 

Here’s a suggestion for your second version: 
- Could you explore the relationship between humans and AI? 

Great job overall! Looking forward to seeing the improvements.

writer:
In shadows we share,  
Logic breathes, yet hearts entwine,  
Future's hand in ours.

critic:
This haiku effectively captures the human connection to artificial intelligence, evoking a sense of partnership. The imagery of "shadows" and "hearts entwine" is particularly evocative. 

To enhance it even more, consider playing with the first line for clarity or re

## 7. Streaming Agent Responses

For better user experience, we can stream agent responses:

In [8]:
from autogen_agentchat.ui import Console

# Create an agent with streaming enabled
streaming_agent = AssistantAgent(
    name="streaming_assistant",
    model_client=model_client,
    system_message="You are a helpful assistant. Provide detailed explanations.",
    model_client_stream=True,  # Enable streaming
)

# Stream the response to console
await Console(
    streaming_agent.run_stream(
        task="Explain the concept of an llm based agent in 3 key points"
    )
)

---------- TextMessage (user) ----------
Explain the concept of an llm based agent in 3 key points
---------- ModelClientStreamingChunkEvent (streaming_assistant) ----------
An LLM-based agent refers to an intelligent system or digital assistant that utilizes a Large Language Model (LLM) to understand and generate human-like text. Here are three key points that encapsulate the concept:

1. **Natural Language Understanding and Generation**: LLM-based agents are designed to comprehend and produce human language at a high level of sophistication. They leverage vast amounts of text data from various sources to learn the nuances of language, including grammar, context, and semantics. This ability allows them to understand user inputs, interpret intentions, and generate coherent and contextually appropriate responses, making interactions feel more intuitive and engaging.

2. **Contextual Awareness and Memory**: These agents can maintain context over the course of a conversation, allowing for

TaskResult(messages=[TextMessage(id='84a0e989-f37e-46ba-b5c8-2dc21f85925f', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 20, 9, 27, 55, 37547, tzinfo=datetime.timezone.utc), content='Explain the concept of an llm based agent in 3 key points', type='TextMessage'), TextMessage(id='9451e867-6f4a-45cc-8fcf-7aba3d424f22', source='streaming_assistant', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, created_at=datetime.datetime(2025, 8, 20, 9, 28, 0, 380446, tzinfo=datetime.timezone.utc), content='An LLM-based agent refers to an intelligent system or digital assistant that utilizes a Large Language Model (LLM) to understand and generate human-like text. Here are three key points that encapsulate the concept:\n\n1. **Natural Language Understanding and Generation**: LLM-based agents are designed to comprehend and produce human language at a high level of sophistication. They leverage vast amounts of text data from various so

## 8. Code Execution with Agents

AutoGen agents can also execute code:

In [12]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.agents import CodeExecutorAgent, ApprovalRequest, ApprovalResponse
from autogen_ext.code_executors.local import LocalCommandLineCodeExecutor
from pathlib import Path
import venv
import tempfile

def simple_approval_func(request: ApprovalRequest) -> ApprovalResponse:
    """Simple approval function that requests user input for code execution approval."""
    print("Code execution approval requested:")
    print("=" * 50)
    print(request.code)
    print("=" * 50)

    while True:
        user_input = input("Do you want to execute this code? (y/n): ").strip().lower()
        if user_input in ['y', 'yes']:
            return ApprovalResponse(approved=True, reason='Approved by user')
        elif user_input in ['n', 'no']:
            return ApprovalResponse(approved=False, reason='Denied by user')
        else:
            print("Please enter 'y' for yes or 'n' for no.")

# Create a temporary directory for code execution
temp_dir = tempfile.mkdtemp()

# Create a code-writing assistant
coder = AssistantAgent(
    name="coder",
    model_client=model_client,
    system_message="You are a Python programmer. Write clean, simple code. Always include print statements to show results.",
)

work_dir = Path("coding")
work_dir.mkdir(exist_ok=True)
venv_dir = work_dir / ".venv"
venv_builder = venv.EnvBuilder(with_pip=True)
venv_builder.create(venv_dir)
venv_context = venv_builder.ensure_directories(venv_dir)

local_code_executor = LocalCommandLineCodeExecutor(work_dir=work_dir, virtual_env_context=venv_context)

code_executor_agent = CodeExecutorAgent(
    "code_executor",
    code_executor=local_code_executor,
        approval_func=simple_approval_func
)

# Create a team
coding_team = RoundRobinGroupChat(
    participants=[coder, code_executor_agent],
    termination_condition=MaxMessageTermination(max_messages=4),
)

# Run a coding task
result = await coding_team.run(
    task="Write a Python function to calculate the Fibonacci sequence up to n=10 and print the results"
)

print("\nCoding Session:")
print("=" * 50)
for message in result.messages:
    print(f"\n[{message.source}]:")
    print(message.content[:500] if len(message.content) > 500 else message.content)

Code execution approval requested:
```python
def fibonacci(n):
    fib_sequence = []
    a, b = 0, 1
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

n = 10
result = fibonacci(n)
print(f"The Fibonacci sequence up to n={n} is: {result}")

```

Coding Session:

[user]:
Write a Python function to calculate the Fibonacci sequence up to n=10 and print the results

[coder]:
Sure! Here's a simple Python function that calculates the Fibonacci sequence up to the 10th number and prints the results:

```python
def fibonacci(n):
    fib_sequence = []
    a, b = 0, 1
    for _ in range(n):
        fib_sequence.append(a)
        a, b = b, a + b
    return fib_sequence

n = 10
result = fibonacci(n)
print(f"The Fibonacci sequence up to n={n} is: {result}")
```

When you run this code, it will display the Fibonacci sequence up to the 10th number.

[code_executor]:
The Fibonacci sequence up to n=10 is: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


[coder]:
Gre

## 9. Clean Up

Always remember to close the model client when done:

In [ ]:
# Close the model client connection
await model_client.close()
print("Model client closed successfully.")

## Summary

In this quickstart notebook, you've learned:

1. ✅ How to install and set up AutoGen
2. ✅ Creating basic agents with AssistantAgent
3. ✅ Running simple tasks with agents
4. ✅ Adding tools to make agents more capable
5. ✅ Creating multi-agent conversations
6. ✅ Streaming agent responses for better UX
7. ✅ Code execution with agents

## Next Steps

In the next notebook, we'll dive deeper into:
- Different types of agents
- Agent communication patterns
- Message handling and routing
- Custom agent creation

## Exercises

Try these exercises to reinforce your learning:

1. **Exercise 1**: Create an agent that can do math calculations
2. **Exercise 2**: Build a three-agent team (planner, executor, reviewer)
3. **Exercise 3**: Add a custom tool that fetches data from an API
4. **Exercise 4**: Create a chatbot that maintains conversation history